# Super Tuesday  - Multiple Site Data Analysis Tool

This tool creates a dataframe with "index = site" and "columns = countdates" for each site and each year of the Super Tuesday Survey.

The contents of this data frame are determined by preferences (similar to the 'Super Tuesday Single Site Data analysis tool'.

The 'site' index values are unique, and using the Easting / Northing GPS values in count_location_details.csv the output of this tool can be used to map commuter bicycle traffic patterns in Moreland.

(Or at least that's the result I'm hoping for if I can get it work)

In [16]:
# Configuration
allmoves = ['north_turn_left','north_through','north_turn_right',\
             'east_turn_left','east_through', 'east_turn_right',\
             'south_turn_left','south_through','south_turn_right',\
             'west_turn_left','west_through','west_turn_right']

movementsofinterest = allmoves
goodcols = ['countsite', 'time', 'gender'] + movementsofinterest

reporting = '7to9'
gender = 'allriders'

datadir = './script_output/count_observations/'

In [17]:
import pandas as pd
import glob
import datetime as dt

sites = datadir + '*'

for f in glob.glob(sites):
    sitedir = f +'/*'
    for g in glob.glob(sitedir):
        obscsv = g + '/*'
        for h in glob.glob(obscsv):
            df = pd.read_csv(h, sep=', ', header = 0, usecols = goodcols, parse_dates=[0], \
                               infer_datetime_format=True, engine = 'python')
            
            site = df['countsite'][0]
            
            countdate = df['time'][0]
            countdate = dt.datetime.strptime(countdate,  "%Y-%m-%d %H:%M:%S")
            countdate = countdate.date()
        
            # Sum the counts - this code is a place holder 
            # THis produces an 'all movements' result, TODO, support gender subsetting and peak hour in survey.
            result = df.sum(axis=1,numeric_only=True)
            result = result.sum()

            print site, countdate, result
        

B-BarrowStAlbionSt 2011-03-01 101
B-BarrowStAlbionSt 2013-03-05 112
B-BarrowStAlbionSt 2017-03-07 165
B-EwingStBrunswickRd 2011-03-01 209
B-EwingStBrunswickRd 2013-03-05 293
B-EwingStBrunswickRd 2016-03-01 287
B-FraserStAlbionSt 2012-03-06 68.0
B-FraserStAlbionSt 2014-03-04 84.0
B-FraserStAlbionSt 2017-03-07 114
B-GranthamDawsonSt 2013-03-05 185.0
B-GranthamDawsonSt 2015-03-03 187.0
B-GranthamDawsonSt 2016-03-01 180.0
B-GrayBrunswick 2011-03-01 44.0
B-GrayBrunswick 2013-03-05 62.0
B-SydneyRdAlbionSt 2012-03-06 223
B-SydneyRdAlbionSt 2014-03-04 216
B-SydneyRdAlbionSt 2016-03-01 233
B-SydneyRdBlythSt 2011-03-01 224.0
B-SydneyRdBlythSt 2013-03-05 231.0
B-SydneyRdBlythSt 2017-03-07 208.0
B-SydneyRdBrunswickRd 2011-03-01 463
B-SydneyRdBrunswickRd 2014-03-04 689
B-SydneyRdBrunswickRd 2017-03-07 549
B-SydneyRdGlenlyonRd 2008-03-11 0.0
B-SydneyRdGlenlyonRd 2010-03-02 458.0
B-SydneyRdGlenlyonRd 2012-03-06 281.0
B-SydneyRdGlenlyonRd 2016-03-01 459
B-SydneyRdParkSt 2003-04-01 491
B-SydneyRdParkSt